In [2]:
import numpy as np

In [6]:
def sqrt_method(a : np.ndarray[np.ndarray], b : np.ndarray, show_annotation : bool = False) -> np.ndarray:
    a = np.array(a, float)
    b = np.array(b, float)
    
    if a.shape[0] != a.shape[1]:
        raise ValueError("Matrix a must be square")
    
    if a.shape[0] != b.shape[0]:
        raise ValueError("The dimensions of the two arrays must match")
    
    if not np.array_equal(a, a.T):
        raise ValueError("Matrix a must be symmetric")
    
    det_a = np.linalg.det(a)
    if abs(det_a) < 1e-13:
        raise ValueError("Determinant is zero — system has no unique solution")
    
    n = len(b)
    
    if show_annotation:
        print("System:\n", a, b, "\n")
        
    s = np.zeros((n, n), dtype='complex')
    s[0, 0] = np.sqrt(a[0,0])
    s[0, 1:] = a[0, 1:] / s[0, 0]
    
    for i in range(1, n):
        s[i, i] = np.sqrt(a[i, i] - np.sum(s[:i, i] ** 2) )
        
        for j in range(i+1, n):
            s[i, j] = (a[i, j] - np.sum([s[k, i] * s[k, j] for k in range(i)]) ) / s[i, i]
            
        s[i, :i] = 0
    
    y = np.zeros(n, dtype='complex')
    y[0] = b[0] / s[0, 0]
    
    for i in range(1, n):
        y[i] = (b[i] - np.sum(s[:i, i] * y[:i])) / s[i, i]
    
    if show_annotation:
        print("S: ", s)
        print("\ny: ", y)
        
    x = np.zeros(n, dtype='complex')
    x[n-1] = y[n-1] / s[n-1, n-1]
    
    for i in range(n - 1, -1, -1):
        x[i] = (y[i] - np.sum(s[i, i+1:] * x[i+1:])) / s[i, i]
    
    if show_annotation:    
        print("\nx: ", x)
    
    return x

In [7]:
a = np.array([[1, 3, -2, 0, -2], [3, 4, -5, 1, -3], [-2, -5, 3, -2, 2], [0, 1, -2, 5, 3], [-2, -3, 2, 3, 4]])
b = np.array([0.5, 5.4, 5, 7.5, 3.3])

x = sqrt_method(a, b, show_annotation=True)

b_ = a @ x
max_error = np.max(np.abs(b - b_))
print("max error: ", max_error)

System:
 [[ 1.  3. -2.  0. -2.]
 [ 3.  4. -5.  1. -3.]
 [-2. -5.  3. -2.  2.]
 [ 0.  1. -2.  5.  3.]
 [-2. -3.  2.  3.  4.]] [0.5 5.4 5.  7.5 3.3] 

S:  [[ 1.        +0.j          3.        +0.j         -2.        +0.j
   0.        +0.j         -2.        +0.j        ]
 [ 0.        +0.j          0.        +2.23606798j  0.        -0.4472136j
   0.        -0.4472136j   0.        -1.34164079j]
 [ 0.        +0.j          0.        +0.j          0.        +0.89442719j
   0.        +2.01246118j  0.        +1.56524758j]
 [ 0.        +0.j          0.        +0.j          0.        +0.j
   3.04138127+0.j          2.21938633+0.j        ]
 [ 0.        +0.j          0.        +0.j          0.        +0.j
   0.        +0.j          0.        +0.82199494j]]

y:  [ 0.5       +0.j          0.        -1.74413302j  0.        -7.58027044j
 -2.29336587+0.j          0.        +0.16439899j]

x:  [-6.1+0.j -2.2-0.j -6.8-0.j -0.9+0.j  0.2+0.j]
max error:  4.440892098500626e-15


In [57]:
def return_max_error(a, b):
    x = sqrt_method(a, b)
    b_ = a @ x
    max_error = np.max(np.abs(b - b_))
    return max_error


a2 = np.array([[4, 2],
               [2, 3]])
b2 = np.array([6, 8])

a3 = np.array([[6, 2, 1],
               [2, 5, 2],
               [1, 2, 4]])
b3 = np.array([9, 10, 7])

a4 = np.array([[10, 2, 3, 1],
               [2, 8, 1, 0],
               [3, 1, 9, 4],
               [1, 0, 4, 7]])
b4 = np.array([12, 9, 11, 8])

a5 = np.array([[9, 1, 2, 3, 1],
               [1, 7, 1, 2, 0],
               [2, 1, 8, 1, 2],
               [3, 2, 1, 10, 3],
               [1, 0, 2, 3, 6]])
b5 = np.array([17, 13, 15, 18, 10])

a6 = np.array([[12, 3, 2, 1, 0, 2],
               [3, 9, 1, 0, 2, 1],
               [2, 1, 11, 4, 1, 2],
               [1, 0, 4, 10, 2, 3],
               [0, 2, 1, 2, 8, 1],
               [2, 1, 2, 3, 1, 9]])
b6 = np.array([15, 12, 18, 14, 10, 16])

def generate_spd_matrix(n):
    A = np.random.randn(n, n)
    return np.dot(A.T, A) + n * np.eye(n)


systems = [
    (a2, b2),
    (a3, b3),
    (a4, b4),
    (a5, b5),
    (a6, b6)
]

for n in [8, 12, 16, 20, 40, 60, 80, 100, 200, 400, 600, 800]:
    A = generate_spd_matrix(n)
    x_true = np.ones(n)
    b = A @ x_true
    systems.append((A, b))

for a, b in systems:
    err = return_max_error(a, b)
    print(f"{a.shape[0]}x{a.shape[0]} max error = {err:.3e}")

2x2 max error = 1.776e-15
3x3 max error = 1.776e-15
4x4 max error = 1.776e-15
5x5 max error = 3.553e-15
6x6 max error = 3.553e-15
8x8 max error = 7.105e-15
12x12 max error = 7.105e-15
16x16 max error = 1.421e-14
20x20 max error = 2.132e-14
40x40 max error = 5.684e-14
60x60 max error = 1.279e-13
80x80 max error = 1.705e-13
100x100 max error = 1.990e-13
200x200 max error = 7.958e-13
400x400 max error = 1.364e-12
600x600 max error = 2.899e-12
800x800 max error = 7.276e-12
